In [1]:
import pandas as pd
import numpy as np

In [2]:
orthologs = pd.read_csv('../../data/ref/four_species_orthologous_protein_coding_genes_1to1_filtered.txt', sep='\t')

In [3]:
species = !ls ../../data/preprocessed/gex_counts_by_sample/

In [4]:
clusts = open('../../data/ref/cluster_list.txt').read().split('\n')

In [5]:
f_format = '../../data/preprocessed/gex_counts_by_sample/{0}/{0}_reps_{1}_rna_counts_all_genes.txt'

In [6]:
f_out = '../../data/processed/gex_for_edger/{}_all_species_orthologous_counts.txt'

In [7]:
def subset_orthologs(df_, ortholog_table, specie):
    df_use = df_.reindex(ortholog_table[specie + '_gene'])
    df_use.index = ortholog_table['human_gene']
    return df_use

In [8]:
def save_list(outfile, listlike):
    '''
    writes a file as a newline seperated list, useful for gsea, or storing information in general
    
    args:
        outfile : a file path to write a list of genes
        listline : an interatable object like a python list, or a numpy array
    
    '''
    return open(outfile, 'w').write(''.join([i + '\n' for i in listlike]).strip('\n'))

In [9]:
missing_orthologs = {}

In [10]:
for clust in clusts:
    sample_order = []
    dfs = []
    for specie in species:
        df = pd.read_csv(f_format.format(specie, clust),
                         header = None,
                         sep = '\t').set_index(0)
        if specie =='macaque':
            # print(len(df.index[df.isna().sum(axis=1) >= 1]))
            df = df.loc[~df.index.duplicated(keep='first')]
            df.index = [i.replace('_', '-') for i in df.index.values]
        sample_order += [specie for _ in range(df.shape[1])]
        df_use = subset_orthologs(df_=df, 
                                    ortholog_table=orthologs,
                                    specie=specie)
        
        dfs.append(df_use)
        missing_orthologs[specie] = df_use.index[df_use.isna().sum(axis=1) >= 1]
        
    clust_df = pd.concat(dfs, axis=1)
    clust_df.to_csv(f_out.format(clust), header=None, sep='\t')      
    save_list('../../data/processed/gex_for_edger/sample_order_{}.txt'.format(clust), sample_order)    

In [11]:
[(i[0], len(i[1])) for i in missing_orthologs.items()]

[('human', 189), ('macaque', 71), ('marmoset', 82), ('mouse', 198)]

In [12]:
missing = np.unique(np.concatenate([i for i in missing_orthologs.values()]))

In [13]:
print('{} missing genes, {} orthologs, {} genes profiled in each species'.format(
    len(missing),
    orthologs.shape[0],
    orthologs.shape[0] - len(missing)
    ))

300 missing genes, 15686 orthologs, 15386 genes profiled in each species


In [14]:
def save_list(outfile, listlike):
    '''
    writes a file as a newline seperated list, useful for gsea, or storing information in general
    
    args:
        outfile : a file path to write a list of genes
        listline : an interatable object like a python list, or a numpy array
    
    '''
    return open(outfile, 'w').write(''.join([i + '\n' for i in listlike]).strip('\n'))

In [15]:
save_list('../../data/ref/missing_annot_in_species.txt', missing)

2020